### Import Libraries

In [1]:
import re
import string
import pandas as pd

from sklearn.utils import shuffle

from tqdm import tqdm
from itertools import product

In [2]:
Paths = {
        'Data': './../../Data/',
        'Train': './../../Data/Text2SqlData/train.csv',
        'Validation': './../../Data/Text2SqlData/validation.csv',
        'Test': './../../Data/Text2SqlData/test.csv',
        'New_Train': './../../Data/PreparedText2SQL/train.csv',
        'New_Validation': './../../Data/PreparedText2SQL/validation.csv',
        'New_Test': './../../Data/PreparedText2SQL/test.csv',
    }

df_train = pd.read_csv(Paths['Train'])
df_val = pd.read_csv(Paths['Validation'])
df_test = pd.read_csv(Paths['Test'])

print('Train Length: {}'.format(len(df_train)))
print('Validation Length: {}'.format(len(df_val)))
print('Test Length: {}'.format(len(df_test)))

Train Length: 533
Validation Length: 125
Test Length: 131


In [3]:
df_train

,base_question,folded_questions,query
0,Count of patients grouped by race.,How many <SYN-ARG-patients/people/persons/indi...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
1,Count of patients grouped by race.,<SYN-ARG-calculate/what's/what is/tell me> the...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
2,Count of patients grouped by race.,How many <SYN-ARG-patients/people/persons/indi...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
3,Count of patients grouped by race.,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
4,Count of patients grouped by race.,<SYN-ARG-calculate/tell me> how many <SYN-ARG-...,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
...,...,...,...
528,counts of patients with taking drug <ARG-DRUG>...,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
529,counts of patients with taking drug <ARG-DRUG>...,<SYN-ARG-calculate/tell me> how many <SYN-ARG-...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
530,counts of patients with taking drug <ARG-DRUG>...,<SYN-ARG-number/count/counts> of <SYN-ARG-pati...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
531,counts of patients with taking drug <ARG-DRUG>...,<SYN-ARG-calculate/what's/what is/tell me> the...,SELECT DATE_PART_YEAR(drug_exposure_start_date...


In [4]:
df_train.rename(columns = {'query':'query_generated'}, inplace = True)
df_val.rename(columns = {'query':'query_generated'}, inplace = True)
df_test.rename(columns = {'query':'query_generated'}, inplace = True)

In [5]:
df_train.dtypes

base_question       object
folded_questions    object
query_generated     object
dtype: object

In [6]:
print('Total Base Questions:')
print(f'Train: {df_train.base_question.nunique()}, Val: {df_val.base_question.nunique()}, Test: {df_test.base_question.nunique()}\n')
print('Total Folded Questions:')
print(f'Train: {df_train.folded_questions.nunique()}, Val: {df_val.folded_questions.nunique()}, Test: {df_test.folded_questions.nunique()}\n')
print('Total Query:')
print(f'Train: {df_train.query_generated.nunique()}, Val: {df_val.query_generated.nunique()}, Test: {df_test.query_generated.nunique()}\n')

Total Base Questions:
Train: 56, Val: 56, Test: 56

Total Folded Questions:
Train: 528, Val: 125, Test: 131

Total Query:
Train: 56, Val: 56, Test: 56



In [7]:
uniqueEntities = set()
for i, f in df_train.iterrows():
    sentence = f.loc['base_question']
    words = re.findall(r'<ARG-(.*?)><.>',sentence)
    for w in words:
        uniqueEntities.add(w)
uniqueEntities

{'AGE',
 'CONDITION',
 'DRUG',
 'ETHNICITY',
 'GENDER',
 'RACE',
 'STATE',
 'TIMEDAYS',
 'TIMEYEARS'}

In [8]:
df_train['base_question'][0]

'Count of patients grouped by race.'

In [10]:
df_train['folded_questions'][0]

'How many <SYN-ARG-patients/people/persons/individuals/subjects> <SYN-ARG-are in/fit into/end up in/were in/fitted into/ended up in> each race group?'

In [11]:
def getSentences(s):
    def getAlphabetsList():
        alphabet_string = string.ascii_uppercase
        alphabet_list = list(alphabet_string)
        return alphabet_list

    def getCombinations(s, alphabet_list):
        options = re.findall(r'<SYN-ARG-(.*?)>',s)
        words = [w.split('/') for w in options]

        columnNumber = len(words)
        alphabet_list = alphabet_list[:columnNumber]

        df = pd.DataFrame(words)
        df = df.transpose()
        df.columns = alphabet_list

        cartesianList = [df[alphabet_list[i]] for i in range(columnNumber)]
        combinations  = list(product(*cartesianList))
        combinations = list(filter(lambda x: None not in x, combinations)) # Remove pair with None
        return combinations, columnNumber

    def getPermutedSentences(s, combinations, columnNumber):
        SENTENCES = []
        options = re.sub(r'<SYN-ARG-(.*?)>', '#######',s)
        hashes = re.findall(r'#######', options)
        assert len(hashes) == columnNumber
        for c in combinations:
            for num_cols in range(columnNumber):
                options = options.replace('#######', c[num_cols], 1)
            SENTENCES.append(options)
            options = re.sub(r'<SYN-ARG-(.*?)>', '#######',s)
        assert len(combinations) == len(SENTENCES)
        return SENTENCES

    alphabet_list = getAlphabetsList()
    combinations, columnNumber = getCombinations(s, alphabet_list)
    sentences = getPermutedSentences(s, combinations, columnNumber)
    return sentences

s = 'How many <SYN-ARG-patients/people/persons/individuals/subjects> <SYN-ARG-are in/fit into/end up in/were in/fitted into/ended up in> each race group?'
getSentences(s)

['How many patients are in each race group?',
 'How many patients fit into each race group?',
 'How many patients end up in each race group?',
 'How many patients were in each race group?',
 'How many patients fitted into each race group?',
 'How many patients ended up in each race group?',
 'How many people are in each race group?',
 'How many people fit into each race group?',
 'How many people end up in each race group?',
 'How many people were in each race group?',
 'How many people fitted into each race group?',
 'How many people ended up in each race group?',
 'How many persons are in each race group?',
 'How many persons fit into each race group?',
 'How many persons end up in each race group?',
 'How many persons were in each race group?',
 'How many persons fitted into each race group?',
 'How many persons ended up in each race group?',
 'How many individuals are in each race group?',
 'How many individuals fit into each race group?',
 'How many individuals end up in each race

In [12]:
def getDataFrames(df_type):
    FOLDED, BASE, QUERY = [], [], []
    for i, f in tqdm(df_type.iterrows(), total=df_type.shape[0]):
        folded_qsts = df_type.loc[i]['folded_questions']
        base_qsts = df_type.loc[i]['base_question']
        query_gen = df_type.loc[i]['query_generated']
        sentences = getSentences(folded_qsts)
        FOLDED += sentences
        BASE += [base_qsts]*len(sentences)
        QUERY += [query_gen]*len(sentences)
        assert len(FOLDED) == len(BASE) == len(QUERY)
    
    df = pd.DataFrame()
    df['Base_Question'] = pd.Series(BASE)
    df['Folded_Question'] = pd.Series(FOLDED)
    df['Query_Generated'] = pd.Series(QUERY)
    
    return df

In [13]:
new_df = getDataFrames(df_test)

100%|███████████████████████████████████████████████| 131/131 [00:01<00:00, 77.94it/s]


In [14]:
new_df

,Base_Question,Folded_Question,Query_Generated
0,Count of patients grouped by race.,number of patients in each race group.,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
1,Count of patients grouped by race.,number of patients for each race group.,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
2,Count of patients grouped by race.,number of people in each race group.,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
3,Count of patients grouped by race.,number of people for each race group.,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
4,Count of patients grouped by race.,number of persons in each race group.,"SELECT race, COUNT(DISTINCT pe1.person_id) AS ..."
...,...,...,...
56926,counts of patients with taking drug <ARG-DRUG>...,tell me the counts of patients taking <ARG-DRU...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
56927,counts of patients with taking drug <ARG-DRUG>...,tell me the counts of people taking <ARG-DRUG>...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
56928,counts of patients with taking drug <ARG-DRUG>...,tell me the counts of persons taking <ARG-DRUG...,SELECT DATE_PART_YEAR(drug_exposure_start_date...
56929,counts of patients with taking drug <ARG-DRUG>...,tell me the counts of individuals taking <ARG-...,SELECT DATE_PART_YEAR(drug_exposure_start_date...


In [15]:
shuffled = shuffle(new_df)
new_df = shuffled.reset_index(drop=True)
new_df

,Base_Question,Folded_Question,Query_Generated
0,"Counts of patients taking drug <ARG-DRUG><0>, ...",what's the counts of individuals got <ARG-DRUG...,SELECT COUNT( DISTINCT dr1.person_id) FROM (((...
1,Counts of patients with condition <ARG-CONDITI...,calculate the number of patients with all of t...,SELECT COUNT( DISTINCT a.person_id ) FROM ((SE...
2,Counts of patients with condition <ARG-CONDITI...,what's the counts of individuals with all of t...,SELECT COUNT( DISTINCT a.person_id ) FROM ((((...
3,How many people were treated by drug <ARG-DRUG...,How many subjects receive medication <ARG-DRUG...,SELECT COUNT( DISTINCT con1.person_id ) FROM (...
4,How many people have Condition <ARG-CONDITION>...,what's the count of people suffered from <ARG-...,SELECT COUNT( DISTINCT con1.person_id ) FROM (...
...,...,...,...
56926,Counts of patients with condition <ARG-CONDITI...,What is the number of individuals suffer from ...,SELECT COUNT( DISTINCT person_id) FROM (<SCHEM...
56927,Counts of patients with condition <ARG-CONDITI...,what's the count of persons with all of these ...,SELECT COUNT( DISTINCT a.person_id ) FROM ((SE...
56928,Counts of patients taking drug <ARG-DRUG><0> o...,How many people have treatment with <ARG-DRUG>...,SELECT COUNT( DISTINCT dr1.person_id) FROM (<S...
56929,How many people have Condition <ARG-CONDITION>...,tell me the counts of patients have condition ...,SELECT COUNT( DISTINCT con1.person_id ) FROM (...


#### Before
Train Length: 533  
Validation Length: 125  
Test Length: 131  

In [16]:
df_train = pd.read_csv(Paths['New_Train'])
df_val = pd.read_csv(Paths['New_Validation'])
df_test = pd.read_csv(Paths['New_Test'])

print('Train Length: {}'.format(len(df_train)))
print('Validation Length: {}'.format(len(df_val)))
print('Test Length: {}'.format(len(df_test)))

Train Length: 596961
Validation Length: 145368
Test Length: 56931
